# USD Tutorial: Prims, Attributes and Metadata

**To run this sample:** click _Runtime_ > _Run all_ from the top menu, or use the <kbd>⌘</kbd>/<kbd>CTRL</kbd>+<kbd>F9</kbd> keyboard shortcut.

## Install the `usd-core` Python package

Install the [`usd-core`](https://pypi.org/project/usd-core/) Python package providing the core USD libraries. Note that it does not provide any of the optional plugins or imaging features from the complete USD distribution.

In [ ]:
! pip install usd-core

# See https://pypi.org/project/usd-core/#history for a list of supported USD
# versions.

In [ ]:
%%file sphere_sample.usda
#usda 1.0
def Sphere "sphere"
{
}

## Tutorial

### Prims
Working with Prims is a more complicated since Prims are extremely powerful objects in USD. Prims are referenced by their path in the stage, which is a string in the form of `/Prim/ChildPrim`. `/` is a special prim known as the root prim in a stage.

To get a reference to a prim at a path use `stage_ref.GetPrimAtPath(path)`:

In [ ]:
from pxr import Sdf, Usd, UsdGeom

stage_ref = Usd.Stage.Open('sphere_sample.usda')

prim = stage_ref.GetPrimAtPath('/sphere')
print(prim.GetName()) # Prints "sphere"
print(prim.GetPrimPath()) # Prints "/sphere"

To define a new prim use `stage_ref.DefinePrim(path)`:

In [ ]:
stage_ref = Usd.Stage.Open('sphere_sample.usda')

prim = stage_ref.DefinePrim('/UnTypedPrim')
print(prim.GetName()) # Prints "UnTypedPrim"

To define a new prim with a type use `stage_ref.DefinePrim(path, type_name)` or you can use your Type's `SomeType.Define(stage_ref, path)` method:

In [ ]:
stage_ref = Usd.Stage.Open('sphere_sample.usda')

prim = stage_ref.DefinePrim('/XformPrim', 'Xform')
# Above we have a Usd.Prim, if we want to access all the Xform's types natively,
# we need to get an Xform instance of our prim
xform = UsdGeom.Xform(prim)

print(xform.GetPath()) # Prints "/XformPrim"

# It is often better to use the Define() method of your type right away, since
# it returns your typed instance rather than a Usd.Prim instance

xform_faster = UsdGeom.Xform.Define(stage_ref, '/AnotherXformPrim')

To delete a prim from the current edit layer (please refer to the [documentation about RemovePrim](https://graphics.pixar.com/usd/docs/api/class_usd_stage.html#ac605faad8fc2673263775b1eecad2955) for details) you can use `stage_ref.RemovePrim(path)`:

In [ ]:
stage_ref = Usd.Stage.Open('sphere_sample.usda')
prim = stage_ref.DefinePrim('/UnTypedPrim')

if stage_ref.RemovePrim('/UnTypedPrim'):
    print('/UnTypedPrim removed')

# If you try to access the prim object, it will still reference path but it is
# expired
if (prim.IsValid()):
    print('{} is valid'.format(prim.GetName()))
else:
    print('{} is not valid'.format(prim.GetName()))
  
# The above will print "Accessed invalid expired prim </UnTypedPrim>"

## Attributes
Attributes are the workhorse of storing actual data inside a Prim. Attributes are often defined as part of [Schemas](https://graphics.pixar.com/usd/docs/USD-Glossary.html#USDGlossary-Schema) to make it easier to access context-relevant data from within an instance of that Type.

For example, `Xform` typed Prims have an attribute called `Purpose` which is used to specify the purpose of an imageable prim. It contains one of the following values: `[default, render, proxy, guide]`

Now, you could get this attribute's value in two ways. One, as a generic `prim_ref.GetAttribute(name)` call, but you would have to know that the exact name of the attribute you want is "purpose", and you wouldn't be able to get any code completion in an IDE that way.

The other way is to use the Xform Schema's exposed function for getting the purpose, which is `xform_ref.GetPurposeAttr()`, which returns the same object, but will be typed in an IDE and does not depend on the underlying string name of the attribute.

Most often after you get an Attribute object, you will want to get the attribute's actual value or set it. That can be done with *attribute_ref.Get()* to retrieve the value, and `attribute_ref.Set(value)` to set the value.

Let's see the code for getting an Attribute reference and getting its value:


In [ ]:
stage_ref = Usd.Stage.Open('sphere_sample.usda')

# Get a reference to the Xform instance as well as a generic Prim instance
xform_ref = UsdGeom.Xform.Define(stage_ref, '/XformPrim')
prim_ref = xform_ref.GetPrim()

# Get an attribute reference (not its value!)
purpose_from_xform_ref = xform_ref.GetPurposeAttr()
purpose_from_prim_ref = prim_ref.GetAttribute('purpose')

print(purpose_from_xform_ref == purpose_from_prim_ref) # Prints "True"

# Prints the actual attribute's value, in this case, one of [default, render,
# proxy, guide], since it is the Xform's actual Purpose attribute
print(purpose_from_xform_ref.Get())

To create an attribute that isn't part of a Type's namespace (or it is, but you want to create the attribute "manually"), you must pass the attribute name and its type to `prim_ref.CreateAttribute(name, type)`.

Otherwise, most Types expose a `Set`-style command, for example `xform_ref.SetPurposeAttr(value)`.

### Working with Attributes

You can call the `prim_ref.CreateAttribute(name, type)` function to create the attribute, and we can use the information above to select a valid `type`. It returns a reference to the attribute created, which we can set with `attribute_ref.Set(value)`, and again we can construct a valid value by looking up the constructor above.

In [ ]:
stage_ref = Usd.Stage.CreateInMemory()

# Create a Usd.Prim
prim_ref = stage_ref.DefinePrim('/Prim')

# Create an attribute reference, using an explicit reference to the type
weight_attr = prim_ref.CreateAttribute('weight', Sdf.ValueTypeNames.Float)

print(weight_attr.Get()) # Prints empty string for default Float values, not 0!

print(weight_attr.Get() == None) # Prints "True"
print(weight_attr.Get() == 0) # Prints "False"

# To set an attribute we use the `attribute_ref.Set(value)` function
weight_attr.Set(42.3)

print(weight_attr.Get()) # Prints "42.3"

# Also, you can chain calls like so
print(prim_ref.GetPrim().GetAttribute('weight').Get()) # Prints "42.3"